In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from openai import OpenAI
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [5]:
import pandas as pd
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
import pickle
import tiktoken

In [190]:
dataset_path = "./Names.csv"
df = pd.read_csv(dataset_path)

In [191]:
df

,Num,Name,Age
0,1,Fahim,40
1,2,Fahim Mohammed,45
2,3,Fahim Mohd,50
3,4,Faheem,40
4,5,Fahim M,45
5,6,Intel,50
6,7,Fam,55
7,8,understanding,40
8,9,understand,45
9,10,Mohammed,50


In [192]:
Name = df[df['Num'].between(1,30)]

In [144]:
Name

,Num,Name,Age
0,1,Fahim,40
1,2,Fahim Mohammed,45
2,3,Fahim Mohd,50
3,4,Faheem,40
4,5,Fahim M,45
5,6,Intel,50
6,7,Fam,55
7,8,understanding,40
8,9,understand,45
9,10,Mohammed,50


In [145]:
def generate_embeddings2(text,model="text-embedding-ada-002"):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [195]:
Name["ada_v2"] = Name["Name"].apply(lambda x: generate_embeddings2 (x, model = 'text-embedding-ada-002'))

In [102]:
data = Name[["Name"]].to_dict("records")

In [103]:
data

[{'Name': 'Fahim'},
 {'Name': 'Fahim Mohammed'},
 {'Name': 'Fahim Mohd'},
 {'Name': 'Intelligent'},
 {'Name': 'Faheem'},
 {'Name': 'Fahim M'},
 {'Name': 'Intel '},
 {'Name': 'Fam'},
 {'Name': 'understanding'},
 {'Name': 'understand'},
 {'Name': 'Mohammed'},
 {'Name': 'mohd'},
 {'Name': 'Judicious'},
 {'Name': 'Fahim'},
 {'Name': 'Sumit'},
 {'Name': 'Sunny'},
 {'Name': 'Chetan'},
 {'Name': 'Sumit'},
 {'Name': 'Manish'},
 {'Name': 'Manisha'}]

In [77]:
pip install nomic


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
from nomic import atlas

In [104]:
project = atlas.map_embeddings (
    embeddings=np.array(list(Name['ada_v2'])),
    data=data,
    id_field='id_'
)

2024-04-22 14:34:27.482 | WARNING  | nomic.atlas:map_embeddings:94 - An ID field was not specified in your data so one was generated for you in insertion order.
2024-04-22 14:34:41.173 | INFO     | nomic.project:_create_project:790 - Creating project `spiffy-fratricide` in organization `fahim.job27`
2024-04-22 14:34:43.869 | INFO     | nomic.atlas:map_embeddings:110 - Uploading embeddings to Atlas.
1it [00:01,  1.51s/it]
2024-04-22 14:34:45.388 | INFO     | nomic.project:_add_data:1422 - Upload succeeded.
2024-04-22 14:34:45.397 | INFO     | nomic.atlas:map_embeddings:129 - Embedding upload succeeded.
2024-04-22 14:34:49.561 | INFO     | nomic.project:create_index:1132 - Created map `spiffy-fratricide` in project `spiffy-fratricide`: https://atlas.nomic.ai/map/bac15702-fd94-4db4-99f5-a8dc85235fe0/25915d88-87cc-4bb5-b8e5-f32bfe80890c
2024-04-22 14:34:49.569 | INFO     | nomic.atlas:map_embeddings:142 - spiffy-fratricide: https://atlas.nomic.ai/map/bac15702-fd94-4db4-99f5-a8dc85235fe0/25

In [147]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=10, to_print=True):
    embedding = get_embedding(
        user_query,
        model="text-embedding-ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


In [203]:
Name_search = "mohd"

In [204]:
res = search_docs(Name, Name_search, top_n=10)

,Num,Name,Age,ada_v2,similarities
10,11,mohd,55,"[-0.009717714041471481, -0.009664611890912056,...",1.000000
2,3,Fahim Mohd,50,"[-0.011398974806070328, -0.006652187556028366,...",0.904955
9,10,Mohammed,50,"[-0.01976589299738407, -0.02073008194565773, -...",0.887418
1,2,Fahim Mohammed,45,"[-0.00886420626193285, 0.00014974437362980098,...",0.876281
0,1,Fahim,40,"[-0.0009897744748741388, -0.001947728684172034...",0.848832
4,5,Fahim M,45,"[0.0025000975001603365, 0.002927068853750825, ...",0.844123
3,4,Faheem,40,"[-0.006094548851251602, -0.0002344374079257249...",0.843842
16,17,Manisha,55,"[0.01391406636685133, 0.00211558910086751, 0.0...",0.818567
23,24,Michal,55,"[-0.0073491851799190044, -0.012339209206402302...",0.815109
15,16,Maneesh,55,"[-0.0009631195571273565, 0.007991530932486057,...",0.813161
